In [1]:
from typing import Mapping
from pandas.core.series import Series
import sqlalchemy
import pandas as pd
from sqlalchemy.orm import sessionmaker
import requests
import json
import datetime
import sqlite3

In [7]:
DATABASE_LOCATION = "sqlite:///my_played_tracks.sqlite"
USER_ID = "level_bastard"
TOKEN = "BQA9vJME46YL8Z9SwWpithRZoB2AKuMT8FlKvrQAx7YgR20hYS5A4b2-xl4_rk9bNUU1c6TQkokI-PnB9mSwmaUa7vId8ZTt2x4WNL_71A_hatR2o1n7iOUJd_Y3gpChlrrwb8uUVASpra5QvcSWrLdo"

def check_data(df: pd.DataFrame) -> bool:
    #Check if the dataframe is empty
    if df.empty:
        print("No songs downloaded. Finishing execution.")
        return False

    # Primary Key Check
    if pd.Series(df["played_at"]).is_unique:
        pass
    else:
        raise Exception("Duplicates Detected!")

    # Check for nulls
    if df.isnull().values.any():
        raise Exception("Null value found")

    # Check that all timestamps are from the previous day.
    yesterday = datetime.datetime.now() - datetime.timedelta(days=1)
    yesterday = yesterday.replace(hour=0, minute=0, second= 0, microsecond= 0)

    timestamps = df["timestamp"].tolist()
    for timestamp  in timestamps:
        if datetime.datetime.strptime(timestamp, "%Y-%m-%d") != yesterday:
            raise Exception("At least one of the returned songs does not come from within the last 24 hours")
# Extract
if __name__ == "__main__":
    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json",
        "Authorization": "Bearer {token}".format(token=TOKEN)
    }

    today = datetime.datetime.now()
    yesterday = today - datetime.timedelta(days=1)
    yesterday_unix_timestamp = int(yesterday.timestamp()) * 1000

    r = requests.get("https://api.spotify.com/v1/me/player/recently-played?after={time}".format(time = yesterday_unix_timestamp), headers=headers)

    data = r.json()

    song_names = []
    artist_names = []
    played_at_list = []
    timestamps = []
    album_type = []
    release_date = []
    duration_ms = []

    for song in data["items"]:
        song_names.append(song["track"]["name"])
        artist_names.append(song["track"]["album"]["artists"][0]["name"])
        played_at_list.append(song["played_at"])
        timestamps.append(song["played_at"][0:10])
        album_type.append(song["track"]["album"]["album_type"])
        release_date.append(song["track"]["album"]["release_date"])
        duration_ms.append(song["track"]["duration_ms"])
        
    song_dict = {
        "song_name" : song_names,
        "artist_name" : artist_names,
        "played_at" : played_at_list,
        "timestamp" : timestamps,
        "album_type" : album_type,
        "release_date" : release_date,
        "running_time" : duration_ms
    }

    song_df = pd.DataFrame(song_dict, columns= ["song_name", "artist_name", "played_at", "timestamp", "album_type", "release_date", "running_time"])
    song_df["running_time"] = pd.to_datetime(song_df["running_time"], format='%M%S')
if check_data(song_df):
    print("Data valid, proceed to Load stage")

ValueError: unconverted data remains: 400

In [ ]:
data

In [8]:
song_df

,song_name,artist_name,played_at,timestamp,album_type,release_date,running_time
0,Rollin Stone,Little Simz,2021-06-22T22:08:00.850Z,2021-06-22,single,2021-06-14,219400
1,BEJOUHHH,Bejo,2021-06-22T22:04:20.981Z,2021-06-22,album,2021-06-18,84123
2,Press Play (Gamer Girl),Cookiee Kawaii,2021-06-22T21:59:50.712Z,2021-06-22,single,2021-06-18,124444
3,Organic Cruis'n 2,Tha Audio Unit,2021-06-22T21:57:46.513Z,2021-06-22,single,2021-06-18,286711
